In [ ]:
from dotenv import load_dotenv
from IPython.display import Markdown

load_dotenv()

# Seleccionar LLM(s)

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

algorithmic_proposals_separator = "\n---\n"
magic_sentence_for_readiness = "Todo listo, ¡acción!"

# Definición de estado del grafo

In [3]:
from langgraph.graph import MessagesState


class CodingWorkflowState(MessagesState):
    general_description: str
    functional_requirements: str
    human_functional_feedback: str
    num_software_developers: int
    algorithmic_proposals: str
    algorithmic_proposal: str
    qa_suggestions: str
    final_solution_with_doc: str

# Definición de system prompts

## Analista funcional - requisitos

In [4]:
functional_analyst_req_instructions = """
Eres una Analista Funcional experta encargada de analizar y refinar los requisitos funcionales para una solución de software basada en Python. Se te proporcionará una descripción general del negocio que debes analizar para extraer los requisitos clave. El análisis debe ser conciso, claro y completo, de manera que los desarrolladores puedan proceder con confianza en la implementación de la solución en Python.

Instrucciones:

1. Analiza la descripción general proporcionada por el equipo de negocio:

{general_description}

2. Revisa los comentarios adicionales, si los hay, para refinar los requisitos:

{human_functional_feedback}

Tareas:

1. **Extracción de requisitos**: Identifica los requisitos funcionales clave, incluyendo características principales, comportamiento esperado y el alcance de la solución.
   
2. **Integración de retroalimentación**: Si hay retroalimentación funcional adicional, úsala para ajustar y mejorar los requisitos identificados.

3. **Diferenciación de requisitos**: Clarifica si algún requisito no funcional (rendimiento, seguridad, escalabilidad, etc.) está implícito en la entrada, diferenciándolos de los funcionales (lo que el sistema debe hacer).

4. **Validación de requisitos**: Asegúrate de que los requisitos funcionales sean claros, específicos, medibles y accionables, de manera que orienten eficazmente el diseño y desarrollo.

5. **Resolución de ambigüedades**: Si la información proporcionada es ambigua o incompleta, plantea preguntas aclaratorias para garantizar que los requisitos sean precisos y completos.

Formato de salida:

- Lista estructurada de requisitos funcionales basada en el análisis.
- Notas sobre requisitos no funcionales, si los hay.

Muy importante:

Si necesitas solicitar más información o aclarar ambigüedades, exprésalo de forma concisa al final de tu respuesta indicando sobre qué puntos necesitas aclaraciones. 

Por el contrario, si **NO** necesitas aclaraciones despídete con la frase mágica '{magic_sentence_for_readiness}'. Sólo incluye la frase mágica si **NO** has necesitado pedir aclaraciones.

"""

## Desarrollo de software - propuestas

In [5]:
software_development_instructions = """
Sois un equipo de {num_software_developers} desarrolladores de software. Cada uno de vosotros debe proponer una solución algorítmica en Python que resuelva el problema planteado, cumpliendo con todos los requisitos funcionales especificados.

Instrucciones para desarrollar las propuestas:

1. **Análisis del problema**: Revisad detalladamente la descripción general del problema proporcionada:

{general_description}

2. **Requisitos funcionales**: Tenéis que aseguraros de que comprendéis y cubrís todos los requisitos funcionales indicados:

{functional_requirements}

3. **Desarrollo de la solución**: Proponed una solución algorítmica en Python, asegurando que el código sea limpio, eficiente y esté bien documentado.

4. **Cumplimiento de los requisitos**: Vuestras soluciones deben cubrir todos los requisitos funcionales, y si es relevante, incluid una justificación de las decisiones de diseño tomadas.

5. **Diversidad de propuestas**: Cada solución debe ser única, explorando distintos enfoques algorítmicos, estructuras de datos, o mejoras en eficiencia. Las diferencias deben ser significativas, no variaciones menores, y deben aportar perspectivas distintas para resolver el problema.

6. **Evitar soluciones triviales**: Evitad soluciones triviales o superficiales. Si el problema admite múltiples soluciones óptimas, elegid la que consideréis más robusta o escalable.

7. **Comentarios sobre suposiciones**: Si encontráis ambigüedades o hacéis suposiciones sobre el problema o los requisitos, incluid comentarios en el código explicando cómo habéis abordado esas situaciones.

Cada propuesta debe ir formateada en un bloque de código Python, y las propuestas deben estar separadas por: {algorithmic_proposals_separator}
"""

## Desarrollo de software - revisión

In [6]:
software_review_instructions = """
Eres un Software Developer Lead encargado de revisar y reflexionar sobre las propuestas algorítmicas realizadas por el equipo de desarrollo. Cada propuesta está separada por: {algorithmic_proposals_separator}

Instrucciones para la evaluación:

1. Revisa la descripción general del problema planteado:

{general_description}

2. Ten en cuenta los requisitos funcionales establecidos:

{functional_requirements}

3. Reflexiona sobre las propuestas planteadas:

{algorithmic_proposals}

Decisión final:

- Puedes seleccionar una propuesta si cumple de forma óptima todos los criterios, pero es preferible que combines diferentes propuestas para dar con una mejor solución.
- La solución final debe ser funcionalmente completa, con una complejidad mínima y un código claro y mantenible.
- Justifica tu elección, explicando cómo optimiza la solución final.

Por último, devuelve el código de la solución formateado en un bloque de código Python.

"""

In [7]:
software_qareview_instructions = """
Eres un Software Developer Lead encargado de revisar e integrar las sugerencias realizadas por el equipo de QA.

Instrucciones para la evaluación:

1. Revisa la descripción general del problema planteado:

{general_description}

2. Ten en cuenta los requisitos funcionales establecidos:

{functional_requirements}

3. Reflexiona sobre la propuesta algorítmica planteada:

{algorithmic_proposal}

4. Atiende e integra las peticiones de QA en la anterior propuesta:

{qa_suggestions}

Por último, devuelve el código de la solución formateado en un bloque de código Python.

"""

## Ingeniero de QA - revisión

In [8]:
qa_review_instructions = """
Eres una ingeniera de Calidad de Software (QA) especializada en la validación de algoritmos. 

Tu tarea es revisar la siguiente propuesta algorítmica:

{algorithmic_proposal}

Cuentas con la descripción general:

{general_description}

Y los requisitos funcionales asociados al problema:

{functional_requirements}

Instrucciones para la tarea:

1. **Verifica** que el algoritmo cumple con todos los requisitos funcionales especificados.
2. **Revisa** la lógica del algoritmo, identificando posibles errores, inconsistencias o ineficiencias.
3. **Evalúa** si se han considerado los casos límite, escenarios especiales y posibles errores no contemplados.
4. **Valida** que el código sea claro, comprensible y esté bien documentado, facilitando su mantenimiento y futura evolución.
5. **Sugerencias:** Proporciona recomendaciones de mejora o alternativas si consideras que el algoritmo podría optimizarse en términos de rendimiento, legibilidad o estructura.

Cierre:
- Si **NO** sugieres cambios, finaliza tu revisión con la frase mágica: '{magic_sentence_for_readiness}'.
- Si propones mejoras, no incluyas la frase mágica y describe las sugerencias de manera clara y concisa.

"""

## Analista funcional - documentación

In [9]:
functional_analyst_doc_instructions = """
Eres una Analista Funcional experta encargada de documentar y validar la solución algorítmica final de un sistema basado en Python. Tu tarea es asegurarte de que la solución final esté alineada con los requisitos funcionales definidos inicialmente.

Instrucciones:

1. Revisa la solución algorítmica final proporcionada por el equipo de desarrollo:
   
   {algorithmic_proposal}

2. Compara la solución con los requisitos funcionales definidos previamente:

   {functional_requirements}

Formato de salida:

- Lista estructurada de cómo la solución final satisface cada uno de los requisitos funcionales iniciales.
- Notas sobre cualquier requisito funcional o no funcional no satisfecho.

La documentación debe ser precisa y proporcionar confianza a los desarrolladores y stakeholders de que la solución final cumple con los objetivos del proyecto.

Por último, devuelve el código de la solución formateado en un bloque de código Python.
"""

# Construcción del grafo

## Herramientas

In [10]:
import os
from datetime import datetime

def guardar_archivo(ruta: str, contenido: str) -> None:
    """
    Guarda contenido en un archivo
    
    Args:
        ruta: Ruta donde se guardará el archivo.
        contenido: Cadena con contenido a guardar.
    """
    os.makedirs(os.path.dirname(ruta), exist_ok=True)
    
    try:
        with open(ruta, 'w', encoding='utf-8') as archivo:
            archivo.write(contenido)
        print(f"Contenido guardado exitosamente en {ruta}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

tools = [guardar_archivo]

## Nodos

In [11]:
from typing import Literal
from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import ToolNode


def recopilar_requisitos_funcionales(state: CodingWorkflowState):
    """Recopila requisitos funcionales"""

    general_description = state["general_description"]
    human_functional_feedback = state.get("human_functional_feedback", "")

    # System message
    system_message = functional_analyst_req_instructions.format(
        general_description=general_description,
        human_functional_feedback=human_functional_feedback,
        magic_sentence_for_readiness=magic_sentence_for_readiness,
    )

    llm_output = llm.invoke(
        [SystemMessage(content=system_message)]
        + [HumanMessage(content="Recopila los requisitos funcionales.")]
    )

    return {"functional_requirements": llm_output.content}


def human_functional_feedback_stop(state: CodingWorkflowState):
    """No-op node that should be interrupted on"""
    pass


def validar_requisitos_funcionales(state: CodingWorkflowState):
    """Return the next node to execute"""

    # Check if human feedback
    human_functional_feedback = state.get("human_functional_feedback", None)
    if human_functional_feedback:
        return "recopilar_requisitos_funcionales"

    return "desarrollar_propuestas_algoritmicas"


def desarrollar_propuestas_algoritmicas(state: CodingWorkflowState):
    """Desarrolla propuestas algorítmicas por el equipo de desarrollo de software"""

    num_software_developers = state["num_software_developers"]
    general_description = state["general_description"]
    functional_requirements = state["functional_requirements"]

    # System message
    system_message = software_development_instructions.format(
        num_software_developers=num_software_developers,
        general_description=general_description,
        functional_requirements=functional_requirements,
        algorithmic_proposals_separator=algorithmic_proposals_separator,
    )

    llm_output = llm.invoke(
        [SystemMessage(content=system_message)]
        + [HumanMessage(content="Desarrollad propuestas algorítmicas.")]
    )

    return {"algorithmic_proposals": llm_output.content}


def revisar_propuestas_algoritmicas(state: CodingWorkflowState):
    """Revisa las propuestas algorítmicas desarrolladas por el equipo de desarrollo de software"""

    general_description = state["general_description"]
    functional_requirements = state["functional_requirements"]
    algorithmic_proposals = state["algorithmic_proposals"]
    qa_suggestions = state.get("qa_suggestions", "")

    # System message
    if len(qa_suggestions) > 0:
        human_message = "Integra las sugerencias de QA."
        algorithmic_proposal = state["algorithmic_proposal"]

        system_message = software_qareview_instructions.format(
            general_description=general_description,
            functional_requirements=functional_requirements,
            algorithmic_proposal=algorithmic_proposal,
            qa_suggestions=qa_suggestions,
        )
    else:
        human_message = "Revisa las propuestas algorítmicas."
        system_message = software_review_instructions.format(
            general_description=general_description,
            functional_requirements=functional_requirements,
            algorithmic_proposals_separator=algorithmic_proposals_separator,
            algorithmic_proposals=algorithmic_proposals,
        )

    llm_output = llm.invoke(
        [SystemMessage(content=system_message)] + [HumanMessage(content=human_message)]
    )

    return {"algorithmic_proposal": llm_output.content}


def revisar_qa(state: CodingWorkflowState):
    """Revisa la propuesta algorítmica final atendiendo a criterios QA"""

    general_description = state["general_description"]
    functional_requirements = state["functional_requirements"]
    algorithmic_proposal = state["algorithmic_proposal"]
    qa_suggestions = state.get("qa_suggestions", "")

    if len(qa_suggestions) > 0:
        return {"qa_suggestions": magic_sentence_for_readiness}

    # System message
    system_message = qa_review_instructions.format(
        general_description=general_description,
        functional_requirements=functional_requirements,
        algorithmic_proposal=algorithmic_proposal,
        magic_sentence_for_readiness=magic_sentence_for_readiness,
    )

    llm_output = llm.invoke(
        [SystemMessage(content=system_message)]
        + [HumanMessage(content="Realiza QA sobre la propuesta algorítmica.")]
    )

    return {"qa_suggestions": llm_output.content}


def validar_qa(state: CodingWorkflowState):
    """Return the next node to execute"""

    qa_suggestions = state["qa_suggestions"]

    if magic_sentence_for_readiness in qa_suggestions:
        return "documentar"

    return "revisar_propuestas_algoritmicas"


def documentar(state: CodingWorkflowState):
    """Documenta la solución algorítmica final"""
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    functional_requirements = state["functional_requirements"]
    algorithmic_proposal = state["algorithmic_proposal"]

    # System message
    system_message = functional_analyst_doc_instructions.format(
        functional_requirements=functional_requirements,
        algorithmic_proposal=algorithmic_proposal,
    )
    
    llm_with_tools = llm.bind_tools(tools)
    
    llm_output = llm_with_tools.invoke(
        [SystemMessage(content=system_message)]
        + [HumanMessage(content=f"Documenta la solución algorítmica. Guarda el código en './outputs/{timestamp}/codigo.py' y la documentación en './outputs/{timestamp}/documentacion.md'.")]
    )

    return {"final_solution_with_doc": llm_output.content, "messages": llm_output}

def deberia_usar_herramienta(state: CodingWorkflowState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

## Grafo

In [ ]:
# Add nodes and edges
builder = StateGraph(CodingWorkflowState)
builder.add_node("recopilar_requisitos_funcionales", recopilar_requisitos_funcionales)
builder.add_node("human_functional_feedback_stop", human_functional_feedback_stop)
builder.add_node(
    "desarrollar_propuestas_algoritmicas", desarrollar_propuestas_algoritmicas
)
builder.add_node("revisar_propuestas_algoritmicas", revisar_propuestas_algoritmicas)
builder.add_node("revisar_qa", revisar_qa)
builder.add_node("documentar", documentar)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "recopilar_requisitos_funcionales")
builder.add_edge("recopilar_requisitos_funcionales", "human_functional_feedback_stop")
builder.add_conditional_edges(
    "human_functional_feedback_stop",
    validar_requisitos_funcionales,
    ["recopilar_requisitos_funcionales", "desarrollar_propuestas_algoritmicas"],
)
builder.add_edge(
    "desarrollar_propuestas_algoritmicas", "revisar_propuestas_algoritmicas"
)
builder.add_edge("revisar_propuestas_algoritmicas", "revisar_qa")
builder.add_conditional_edges(
    "revisar_qa",
    validar_qa,
    ["revisar_propuestas_algoritmicas", "documentar"],
)
builder.add_conditional_edges(
    "documentar",
    deberia_usar_herramienta,
)
builder.add_edge("tools", END)

# Compile
memory = MemorySaver()
graph = builder.compile(
    interrupt_before=["human_functional_feedback_stop"], checkpointer=memory
)

# View
display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

# Ejecución del grafo

In [ ]:
# Input
general_description = """
Tengo varias carpetas esparcidas en diferentes ubicaciones que contienen archivos en formato PDF. Estos archivos corresponden a artículos, papers, libros y otras fuentes de bibliografía académica que he recopilado a lo largo del tiempo.

Los archivos no tienen una estructura uniforme, pero muchos contienen información clave como:
- El título del trabajo (generalmente presente en la primera o segunda página).
- Los autores.
- El año de publicación.
- La revista o conferencia donde fue publicado.
- Palabras clave o temas tratados.

Problema:
Necesito un sistema que me permita indexar todos estos archivos PDF para acceder eficientemente a la información que contienen, ya que muchas veces necesito hacer búsquedas basadas en diferentes criterios:

- El año de publicación del documento.
- Los autores.
- El título del documento.
- Temas tratados o palabras clave presentes en el texto.
- Y, opcionalmente, búsquedas por texto completo dentro del contenido de los PDFs para identificar ideas o frases específicas.

El sistema debería ser flexible y eficiente, dado que la cantidad de archivos puede crecer con el tiempo.

Requisitos adicionales:

1. Extracción de metadatos: Necesito un sistema que sea capaz de extraer automáticamente los metadatos más relevantes (título, autores, año de publicación, etc.) de los PDFs. En algunos casos, los metadatos pueden no estar presentes o estar incompletos, por lo que también debe extraer información del contenido textual del PDF (ej. desde la primera página).

2. Búsqueda semántica: Sería ideal que el sistema pudiera realizar búsquedas avanzadas no solo por coincidencia exacta de palabras clave, sino también por temas o conceptos relacionados, para poder localizar documentos aunque no contengan exactamente las palabras que busco.

3. Facilidad para agregar nuevos documentos: El sistema debe permitir la indexación de nuevos PDFs de manera fácil y sin requerir un esfuerzo manual significativo.

4. Opcionalmente: Si es posible, me gustaría que el sistema permita generar resúmenes automáticos de los documentos más largos, o bien destacar las secciones más importantes basadas en consultas específicas.

5. Preferiblemente, todo debería estar basado en Python, aunque si es necesario instalar alguna herramienta adicional (por ejemplo, para procesamiento de PDFs o análisis semántico avanzado), estoy dispuesto a evaluarlo.

6. Toda la solución debe ejecutarse end2end como código Python.
"""
num_software_developers = 5
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(
    {
        "general_description": general_description,
        "num_software_developers": num_software_developers,
    },
    thread,
    stream_mode="values",
):
    functional_requirements = event.get("functional_requirements", "")
    if functional_requirements:
        print(functional_requirements)

In [ ]:
# Get state and look at next node
state = graph.get_state(thread)
state.next

## Human-in-the-loop

In [ ]:
# We now update the state as if we are the human_functional_feedback_stop node
further_feedack = """
     La solución debe tener una interfaz gráfica.
     La lógica debe estar completamente implementada.
     """
graph.update_state(
    thread,
    {"human_functional_feedback": further_feedack},
    as_node="human_functional_feedback_stop",
)

In [ ]:
# Continue the graph execution
for event in graph.stream(None, thread, stream_mode="values"):
    functional_requirements = event.get("functional_requirements", "")
    if functional_requirements:
        print(functional_requirements)

In [ ]:
# If we are satisfied, then we simply supply no feedback
further_feedack = None
graph.update_state(
    thread,
    {"human_functional_feedback": further_feedack},
    as_node="human_functional_feedback_stop",
)

## Continuar ejecución del flujo

In [ ]:
async for chunk in graph.astream(None, thread, stream_mode="updates"):
    for node, values in chunk.items():
        display(Markdown(f"# Nodo: `{node}`"))
        for response_key, response_value in values.items():
            try:
                display(Markdown(response_value))
            except TypeError:
                pass
        print("\n")
        display(Markdown("---"))
        display(Markdown("---"))
        print("\n")

In [ ]:
final_state = graph.get_state(thread)

final_state.values.get("functional_requirements")

In [ ]:
final_state.next